# 快速入门Blackhole 机器学习功能
---
* Blackhole ML是高性能机器学习引擎，具备单机大数据处理能力，拥有与Scikit-learn一致的接口，享有单机处理的便利性和媲美分布式处理的性能。
* 该指南旨在介绍Blackhole ML模块提供的一些主要功能，如数据预处理、模型训练、预测、评估、调优、持久化等基本操作。


## Blackhole环境准备
**CodeLab平台默认不安装Blackhole，请先到导航左边“包管理”页面安装blackhole。**  
**这个文档简单介绍了Blackhole数据分析的常用接口，更多关于blackhole使用方法和案例，请参考[Blackhole简介和基本用法](https://cloud.baidu.com/doc/BML/s/9khemrnv7)。**

## 模型训练和预测
---
Blackhole ML提供了许多内置的机器学习算法，称为estimator，每个estimator都可以用它的fit方法来拟合数据。
以下是一个简单的例子：使用RandomForestClassifier estimator来拟合一些基本的数据：

In [1]:
import blackhole.dataframe as bhdf
from blackhole.ml.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state=0)
X = bhdf.DataFrame([[1, 2, 3],[11, 12, 13]]) # 2个样本，3个特征
y = bhdf.Series([0, 1])
clf.fit(X, y)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%


fit方法通常接受两个输入参数：
* 样本矩阵X，X的size为（n_samples，n_features），samples代表行，features代表列。
* 标签值y，对于分类任务是integers类型，对于回归任务是real类型数据，对于非监督学习任务不需要指定y；y的第i个元素，对应X的第i个样本的标签值。

一旦拟合了estimator，就可以用来预测新数据的标签值，不需要再重新训练estimator：

In [7]:
clf.predict(bhdf.DataFrame([[4, 5, 6], [14, 15, 16]]))

Parse progress: |█████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
Export File progress: |███████████████████████████████████████████████████| 100%


,predict
0,0
1,1


## 数据转换与预处理
* 为了获取较好的模型效果，在模型训练前，通常会进行数据转换和预处理的操作。
* Blackhole ML提供了fit、transform或者同时fit_transform方法对训练数据进行转换和预处理：

In [9]:
from blackhole.ml.preprocessing import StandardScaler
X = bhdf.DataFrame([[0, 15],[1, -10]])
StandardScaler().fit(X).transform(X)

,0,1
0,-1.0,1.0
1,1.0,-1.0


In [10]:
StandardScaler().fit_transform(X)

,0,1
0,-1.0,1.0
1,1.0,-1.0


更多预处理功能可参考Blackhole ML的preprocessing模块

## 模型评估
模型的拟合效果，需要通过模型评分方法来度量，Blackhole ML提供了针对分类、回归、聚类等不同学习任务的模型评估方法，以准确率方法为例：

In [13]:
from blackhole.ml.linear_model import LogisticRegression
from blackhole.ml.metrics import accuracy_score
X = bhdf.DataFrame([[1, 2, 3],[11, 12, 13]]) # 2个样本，3个特征
y = bhdf.Series([0, 1])
clf = LogisticRegression()
clf.fit(X, y)
y_predict = clf.predict(X)
accuracy_score(y, y_predict)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
glm Model Build progress: |███████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
Export File progress: |███████████████████████████████████████████████████| 100%


1.0

## 模型优化
* Blackhole ML的所有estiator都有可以调整的参数，称为超参数。例如，GradientBoostingRegressor中树的最大深度max_depth。
* estimator的泛化能力通常依赖于超参数的调整。Blackhole ML提供了自动查找最佳超参组合的方法（通过交叉验证实现），如GridSearchCV、RandomizedSearchCV

In [24]:
from sklearn.datasets import fetch_california_housing
from blackhole.ml.ensemble import GradientBoostingRegressor
from blackhole.ml.model_selection import RandomizedSearchCV
from blackhole.ml.model_selection import train_test_split

X, y = fetch_california_housing(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
param_distributions = {'learn_rate': [0.1, 0.01, 0.001], 'max_depth': [2, 4, 6]}
search = RandomizedSearchCV(
    estimator=GradientBoostingRegressor(random_state=0),
    n_iter=5,
    scoring='mse',
    param_distributions=param_distributions,
    random_state=0
)
search.fit(X_train, y_train)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
gbm Grid Build progress: |████████████████████████████████████████████████| 100%


In [26]:
search.best_params

{'learn_rate': 0.1, 'max_depth': 6}

In [27]:
search.best_score

0.19857402550517486

## 模型保存
Blackhole ML提供了save_model、load_model方法用于模型的保存和加载

In [29]:
from blackhole.ml.linear_model import LogisticRegression
from blackhole.ml.metrics import accuracy_score
from blackhole.ml import save_model

X = bhdf.DataFrame([[1, 2, 3],[11, 12, 13]]) # 2个样本，3个特征
y = bhdf.Series([0, 1])
clf = LogisticRegression()
clf.fit(X, y)
save_model(clf, 'lr_model')

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
glm Model Build progress: |███████████████████████████████████████████████| 100%
